# Chapter 1: Prompt Chaining

Key Takeaways: 
- Prompt Chaining breaks down complex tasks into a sequence of smaller, focused steps.
- This is occasionally known as the Pipeline pattern.
- Each step in a chain involves an LLM call or processing logic, using the output of the
previous step as input.
- This pattern improves the reliability and manageability of complex interactions with
language models.
- Frameworks like LangChain/LangGraph, and Google ADK provide robust tools to
define, manage, and execute these multi-step sequences.

## Setup and Initialization

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()
# Make sure your OPENAI_API_KEY is set in the .env file

/opt/anaconda3/envs/asd/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


True

In [2]:
# Initialize the Language Model
llm = ChatOpenAI(temperature=0)

## Pattern 1: Information Processing Workflows

This pattern demonstrates processing raw information through multiple transformations. We'll simulate extracting content from a URL, summarizing it, extracting entities, searching a knowledge base, and generating a final report.

**Use Case**: Document analysis and reporting pipeline

In [5]:
# Pattern 1: Information Processing Workflows
# 5-step chain: extract text -> summarize -> extract entities -> search KB -> generate report

# Step 1: Extract text content (simulated)
prompt_extract_text = ChatPromptTemplate.from_template(
    "Extract and clean the main text content from this document:\n\n{raw_content}"
)

# Step 2: Summarize the cleaned text
prompt_summarize = ChatPromptTemplate.from_template(
    "Provide a concise summary of the following text (2-3 sentences):\n\n{cleaned_text}"
)

# Step 3: Extract entities
prompt_extract_entities = ChatPromptTemplate.from_template(
    "Extract key entities (names, dates, locations, organizations) from this text in JSON format:\n\n{text}"
)

# Step 4: Search knowledge base (simulated with contextual query)
prompt_search_kb = ChatPromptTemplate.from_template(
    "Based on these entities: {entities}, generate 2-3 relevant search queries for a knowledge base."
)

# Step 5: Generate final report
prompt_generate_report = ChatPromptTemplate.from_template(
    """Generate a comprehensive report incorporating:
    
Summary: {summary}
Key Entities: {entities}
Related Searches: {searches}

Format the report in a professional manner with sections."""
)

In [ ]:
# Build the chain
from langchain_core.runnables import RunnablePassthrough

parser = StrOutputParser()

# Step 1: Extract cleaned text
extract_chain = prompt_extract_text | llm | parser

# Step 2: Summarize the cleaned text
summarize_chain = prompt_summarize | llm | parser

# Step 3: Extract entities
entities_chain = prompt_extract_entities | llm | parser

# Step 4: Generate search queries
search_chain = prompt_search_kb | llm | parser

# Full information processing workflow using proper Runnables
info_workflow = (
    # First, extract cleaned text and pass through the raw_content
    RunnablePassthrough.assign(
        cleaned_text=extract_chain
    )
    # Then generate summary and entities based on cleaned_text
    | RunnablePassthrough.assign(
        summary=lambda x: summarize_chain.invoke({"cleaned_text": x["cleaned_text"]}),
        entities=lambda x: entities_chain.invoke({"text": x["cleaned_text"]})
    )
    # Then generate searches based on entities
    | RunnablePassthrough.assign(
        searches=lambda x: search_chain.invoke({"entities": x["entities"]})
    )
    # Finally generate the report using all the collected information
    | prompt_generate_report
    | llm
    | parser
)

In [ ]:
# Execute the workflow
sample_document = """Tesla Inc. announced on March 15, 2024, that CEO Elon Musk will unveil 
the company's new Gigafactory in Austin, Texas. The facility will produce 500,000 electric 
vehicles annually and employ over 10,000 workers. The announcement came during a press 
conference attended by Texas Governor Greg Abbott and Austin Mayor Kirk Watson."""

result = info_workflow.invoke({"raw_content": sample_document})
print("\n=== PATTERN 1: Information Processing Workflow ===")
print(result)


=== PATTERN 1: Information Processing Workflow ===
**Report on Tesla Inc.'s New Gigafactory in Austin, Texas**

**Summary:**
On March 15, 2024, Tesla Inc. announced the unveiling of a new Gigafactory in Austin, Texas. The facility is set to produce 500,000 electric vehicles annually and provide jobs for over 10,000 workers. The announcement was made during a press conference attended by Texas Governor Greg Abbott and Austin Mayor Kirk Watson.

**Key Entities:**
- **Organization:** Tesla Inc.
- **Date:** March 15, 2024
- **Location:** Austin, Texas
- **Persons:** Elon Musk, Greg Abbott, Kirk Watson
- **Facility:** Gigafactory
- **Production Capacity:** 500,000 electric vehicles annually
- **Number of Employees:** Over 10,000

**Related Searches:**
1. **Production Capacity:** The production capacity of Tesla Inc.'s Gigafactory in Austin, Texas is 500,000 electric vehicles annually.
2. **Key Persons:** The key persons involved in the operations of Tesla Inc.'s Gigafactory in Austin, Texa

## Pattern 2: Complex Query Answering

This pattern breaks down complex questions into sub-questions, researches each independently, and synthesizes a comprehensive answer.

**Use Case**: Multi-faceted research questions requiring decomposition

In [11]:
from langchain_core.runnables import RunnablePassthrough

# Step 1: Identify core sub-questions
prompt_identify_subqs = ChatPromptTemplate.from_template(
    """Break down this complex question into 2-3 specific sub-questions:
    
Question: {query}

Provide the sub-questions as a numbered list."""
)

# Step 2: Research the causes
prompt_research_causes = ChatPromptTemplate.from_template(
    """Provide detailed information about the causes of the 1929 stock market crash.
    Include economic, political, and social factors. (3-4 paragraphs)"""
)

# Step 3: Research government response
prompt_research_response = ChatPromptTemplate.from_template(
    """Describe how the U.S. government responded to the 1929 stock market crash.
    Include both immediate actions and longer-term policy changes. (3-4 paragraphs)"""
)

# Step 4: Synthesize the information
prompt_synthesize = ChatPromptTemplate.from_template(
    """Based on the following research, provide a comprehensive answer to the original question:

Original Question: {original_query}

Sub-questions identified:
{subquestions}

Research on Causes:
{causes_info}

Research on Government Response:
{response_info}

Synthesize this into a coherent, well-structured answer."""
)

# Build the complex query chain using RunnablePassthrough
subq_chain = prompt_identify_subqs | llm | parser
causes_chain = prompt_research_causes | llm | parser
response_chain = prompt_research_response | llm | parser

complex_query_chain = (
    # Step 1: Keep original query and identify sub-questions
    RunnablePassthrough.assign(
        original_query=lambda x: x["query"],
        subquestions=subq_chain
    )
    # Step 2 & 3: Add research results (these don't depend on previous steps in this case)
    | RunnablePassthrough.assign(
        causes_info=lambda x: causes_chain.invoke({}),
        response_info=lambda x: response_chain.invoke({})
    )
    # Step 4: Synthesize everything into final answer
    | prompt_synthesize
    | llm
    | parser
)

# Execute
query = "What were the main causes of the stock market crash in 1929, and how did government policy respond?"
result = complex_query_chain.invoke({"query": query})
print("\n=== PATTERN 2: Complex Query Answering ===")
print(result)


=== PATTERN 2: Complex Query Answering ===
The main causes of the stock market crash in 1929, also known as Black Tuesday, were a combination of economic, political, and social factors. The overvaluation of stocks due to speculative buying on margin, the Federal Reserve's raising of interest rates, and the Smoot-Hawley Tariff Act all contributed to the collapse of stock prices. Additionally, a culture of excessive risk-taking and speculation fueled by media and popular culture played a role in the crash.

In response to the crash, the U.S. government took immediate action to stabilize the economy and restore confidence in the financial system. The Emergency Banking Act provided federal assistance to struggling banks, while the Reconstruction Finance Corporation offered loans to stimulate economic activity. Longer-term policy changes included the Securities Act of 1933 and the Securities Exchange Act of 1934 to regulate the securities industry, the Glass-Steagall Act of 1933 to separat

## Pattern 3: Data Extraction and Transformation

This pattern demonstrates iterative extraction and validation, with conditional re-extraction for missing or malformed data.

**Use Case**: Converting unstructured documents to structured data with validation

In [12]:
# Pattern 3: Data Extraction and Transformation
# Iterative extraction with validation and retry logic

import json

# Step 1: Initial extraction attempt
prompt_extract_fields = ChatPromptTemplate.from_template(
    """Extract the following fields from this invoice document and return as JSON:
- invoice_number
- date
- customer_name
- customer_address
- total_amount

Invoice text:
{invoice_text}

Return only valid JSON, no additional text."""
)

# Step 2: Validation and conditional re-extraction
def validate_and_retry(invoice_text):
    extraction_chain = prompt_extract_fields | llm | parser
    
    # First attempt
    result = extraction_chain.invoke({"invoice_text": invoice_text})
    print("\nFirst extraction attempt:")
    print(result)
    
    try:
        data = json.loads(result)
        required_fields = ["invoice_number", "date", "customer_name", "customer_address", "total_amount"]
        missing = [f for f in required_fields if f not in data or not data[f]]
        
        if missing:
            # Retry with specific instructions for missing fields
            print(f"\nMissing fields detected: {missing}")
            print("Attempting re-extraction...")
            
            retry_prompt = ChatPromptTemplate.from_template(
                """The previous extraction was missing these fields: {missing_fields}
                
Please carefully re-examine the invoice and extract these specific fields:
{missing_fields}

Previous extraction:
{previous_result}

Invoice text:
{invoice_text}

Return complete JSON with all fields."""
            )
            
            retry_chain = retry_prompt | llm | parser
            result = retry_chain.invoke({
                "missing_fields": ", ".join(missing),
                "previous_result": result,
                "invoice_text": invoice_text
            })
            print("\nRetry extraction result:")
            print(result)
            
        return result
    except json.JSONDecodeError:
        print("\nInvalid JSON format, requesting reformatting...")
        return result

# Sample invoice
invoice = """ACME Corp Invoice
Invoice #12345
Date: 2024-01-15

Bill To:
John Smith
123 Main Street
New York, NY 10001

Items:
- Laptop Computer (Qty: 2) @ $1,200 = $2,400
- Wireless Mouse (Qty: 5) @ $25 = $125

Subtotal: $2,525
Tax (8%): $202
TOTAL: $2,727
"""

print("=== PATTERN 3: Data Extraction and Transformation ===")
final_data = validate_and_retry(invoice)
print("\n=== Final Validated Data ===")
print(final_data)

=== PATTERN 3: Data Extraction and Transformation ===

First extraction attempt:
{
  "invoice_number": "12345",
  "date": "2024-01-15",
  "customer_name": "John Smith",
  "customer_address": "123 Main Street, New York, NY 10001",
  "total_amount": "$2,727"
}

=== Final Validated Data ===
{
  "invoice_number": "12345",
  "date": "2024-01-15",
  "customer_name": "John Smith",
  "customer_address": "123 Main Street, New York, NY 10001",
  "total_amount": "$2,727"
}


## Pattern 4: Content Generation Workflows

This pattern demonstrates progressive content creation: ideation → selection → outlining → drafting → refinement.

**Use Case**: Structured content creation for articles, reports, or documentation

In [13]:
# Pattern 4: Content Generation Workflows
# 5-step chain: ideas -> selection -> outline -> drafting -> refinement

# Step 1: Generate topic ideas
prompt_generate_ideas = ChatPromptTemplate.from_template(
    """Generate 5 engaging blog post topic ideas about: {interest}
    
Format: numbered list with brief description for each."""
)

# Step 2: Auto-select best idea (or could be user selection)
prompt_select_topic = ChatPromptTemplate.from_template(
    """From these topic ideas, select the most engaging and timely one:

{ideas}

Return only the selected topic title and a brief rationale."""
)

# Step 3: Create detailed outline
prompt_create_outline = ChatPromptTemplate.from_template(
    """Create a detailed outline for a blog post on this topic:

{selected_topic}

Include:
- Introduction
- 3-4 main points with sub-points
- Conclusion"""
)

# Step 4: Draft sections
prompt_draft_section = ChatPromptTemplate.from_template(
    """Write a draft section for this part of the outline:

Section: {section}

Full outline for context:
{full_outline}

Previous sections:
{previous_content}

Write 2-3 paragraphs for this section."""
)

# Step 5: Refine complete draft
prompt_refine = ChatPromptTemplate.from_template(
    """Review and refine this blog post draft for coherence, tone, and grammar:

{complete_draft}

Provide the refined version with improvements."""
)

# Build content generation workflow
ideas_chain = prompt_generate_ideas | llm | parser
select_chain = prompt_select_topic | llm | parser
outline_chain = prompt_create_outline | llm | parser
section_chain = prompt_draft_section | llm | parser
refine_chain = prompt_refine | llm | parser

# Execute workflow (simplified version - full version would iterate through sections)
def content_generation_workflow(interest):
    # Generate and select topic
    ideas = ideas_chain.invoke({"interest": interest})
    print("\n=== Generated Ideas ===")
    print(ideas)
    
    selected = select_chain.invoke({"ideas": ideas})
    print("\n=== Selected Topic ===")
    print(selected)
    
    outline = outline_chain.invoke({"selected_topic": selected})
    print("\n=== Outline ===")
    print(outline)
    
    # Draft first section
    intro = section_chain.invoke({
        "section": "Introduction",
        "full_outline": outline,
        "previous_content": ""
    })
    print("\n=== Introduction Draft ===")
    print(intro)
    
    # For brevity, we'll refine just the intro
    refined = refine_chain.invoke({"complete_draft": intro})
    print("\n=== Refined Introduction ===")
    print(refined)
    
    return refined

print("=== PATTERN 4: Content Generation Workflow ===")
result = content_generation_workflow("artificial intelligence in healthcare")

=== PATTERN 4: Content Generation Workflow ===

=== Generated Ideas ===
1. "The Future of Healthcare: How Artificial Intelligence is Revolutionizing Patient Care"
   - Explore the ways in which AI is transforming the healthcare industry, from personalized treatment plans to predictive analytics for early disease detection.

2. "Ethical Considerations in AI Healthcare: Balancing Innovation with Patient Privacy"
   - Discuss the ethical dilemmas surrounding the use of AI in healthcare, including concerns about data security, patient consent, and bias in algorithms.

3. "From Diagnosis to Treatment: How AI is Streamlining Healthcare Processes"
   - Highlight the efficiency improvements brought about by AI in healthcare, such as faster diagnosis, automated administrative tasks, and optimized treatment plans.

4. "The Role of AI in Mental Health: Breaking Down Barriers to Access and Stigma"
   - Examine how AI is being used to improve mental health care, including virtual therapy sessions, 

## Pattern 5: Conversational Agents with State

This pattern maintains conversation context by building each turn's prompt with accumulated history.

**Use Case**: Chatbots, virtual assistants, multi-turn dialogues

In [14]:
# Pattern 5: Conversational Agents with State
# Maintain context across multiple conversation turns

class ConversationalAgent:
    def __init__(self):
        self.conversation_state = {
            "history": [],
            "user_info": {},
            "intent_stack": []
        }
        
        # Prompt for intent and entity extraction
        self.intent_prompt = ChatPromptTemplate.from_template(
            """Analyze this user message and extract:
1. Primary intent (e.g., 'book_appointment', 'ask_question', 'provide_info')
2. Key entities (names, dates, locations, etc.)

User message: {user_message}

Conversation history: {history}

Return as JSON with 'intent' and 'entities' fields."""
        )
        
        # Prompt for response generation
        self.response_prompt = ChatPromptTemplate.from_template(
            """Generate a helpful response based on:

User message: {user_message}
Detected intent: {intent_info}
Conversation history: {history}
User profile: {user_info}

Be conversational, helpful, and reference previous context when relevant."""
        )
        
        self.intent_chain = self.intent_prompt | llm | parser
        self.response_chain = self.response_prompt | llm | parser
    
    def process_turn(self, user_message):
        # Extract intent and entities
        intent_info = self.intent_chain.invoke({
            "user_message": user_message,
            "history": str(self.conversation_state["history"])
        })
        
        # Update state
        self.conversation_state["history"].append({
            "user": user_message,
            "intent": intent_info
        })
        
        # Generate response
        response = self.response_chain.invoke({
            "user_message": user_message,
            "intent_info": intent_info,
            "history": str(self.conversation_state["history"][:-1]),  # Exclude current
            "user_info": str(self.conversation_state["user_info"])
        })
        
        # Update history with response
        self.conversation_state["history"][-1]["response"] = response
        
        return response

# Demo conversation
print("=== PATTERN 5: Conversational Agent with State ===")
agent = ConversationalAgent()

turns = [
    "Hi, I'd like to book a doctor's appointment",
    "I prefer next Tuesday afternoon",
    "Yes, that works. My name is Sarah Johnson"
]

for i, user_msg in enumerate(turns, 1):
    print(f"\n--- Turn {i} ---")
    print(f"User: {user_msg}")
    response = agent.process_turn(user_msg)
    print(f"Agent: {response}")

print("\n=== Final Conversation State ===")
print(f"Total turns: {len(agent.conversation_state['history'])}")

=== PATTERN 5: Conversational Agent with State ===

--- Turn 1 ---
User: Hi, I'd like to book a doctor's appointment
Agent: Hi there! I'd be happy to help you book a doctor's appointment. Can you please provide me with some more information such as the type of doctor you need to see, your preferred date and time, and any specific location preferences? Let me know so I can assist you further.

--- Turn 2 ---
User: I prefer next Tuesday afternoon
Agent: Hi there! Thank you for letting me know that you prefer next Tuesday afternoon for your doctor's appointment. I will do my best to accommodate that request. Can you please provide me with the type of doctor you need to see and any specific location preferences? This will help me find the best available option for you. Thank you for your cooperation!

--- Turn 3 ---
User: Yes, that works. My name is Sarah Johnson
Agent: Great to meet you, Sarah Johnson! I'm glad to hear that the appointment time works for you. Now that I have your name, is

## Pattern 6: Code Generation and Refinement

This pattern demonstrates iterative code development: requirements → pseudocode → implementation → analysis → refinement.

**Use Case**: AI-assisted programming, code generation tools

In [15]:
# Pattern 6: Code Generation and Refinement
# 5-step chain: requirements -> pseudocode -> code -> analysis -> refinement

# Step 1: Understand requirements and generate pseudocode
prompt_pseudocode = ChatPromptTemplate.from_template(
    """Based on this requirement, write detailed pseudocode:

Requirement: {requirement}

Break down the logic step-by-step in pseudocode format."""
)

# Step 2: Generate initial code
prompt_initial_code = ChatPromptTemplate.from_template(
    """Convert this pseudocode into Python code:

{pseudocode}

Original requirement: {requirement}

Provide clean, well-structured Python code with docstrings."""
)

# Step 3: Analyze for errors and improvements
prompt_analyze_code = ChatPromptTemplate.from_template(
    """Analyze this code for potential errors, edge cases, and improvements:

{code}

List specific issues found:
1. Bugs or logical errors
2. Missing edge case handling
3. Performance issues
4. Code style improvements"""
)

# Step 4: Refine the code
prompt_refine_code = ChatPromptTemplate.from_template(
    """Refine this code based on the identified issues:

Original code:
{code}

Issues to address:
{issues}

Provide the improved version of the code."""
)

# Step 5: Add documentation and tests
prompt_add_docs = ChatPromptTemplate.from_template(
    """Add comprehensive docstrings and 2-3 unit test cases for this code:

{refined_code}

Include:
- Function/class docstrings
- Pytest-style test cases"""
)

# Build code generation workflow
pseudocode_chain = prompt_pseudocode | llm | parser
code_chain = prompt_initial_code | llm | parser
analyze_chain = prompt_analyze_code | llm | parser
refine_chain = prompt_refine_code | llm | parser
docs_chain = prompt_add_docs | llm | parser

def code_generation_workflow(requirement):
    # Generate pseudocode
    pseudocode = pseudocode_chain.invoke({"requirement": requirement})
    print("\n=== Pseudocode ===")
    print(pseudocode)
    
    # Generate initial code
    code = code_chain.invoke({
        "pseudocode": pseudocode,
        "requirement": requirement
    })
    print("\n=== Initial Code ===")
    print(code)
    
    # Analyze for issues
    issues = analyze_chain.invoke({"code": code})
    print("\n=== Code Analysis ===")
    print(issues)
    
    # Refine code
    refined = refine_chain.invoke({
        "code": code,
        "issues": issues
    })
    print("\n=== Refined Code ===")
    print(refined)
    
    # Add documentation
    final = docs_chain.invoke({"refined_code": refined})
    print("\n=== Final Code with Docs and Tests ===")
    print(final)
    
    return final

print("=== PATTERN 6: Code Generation and Refinement ===")
requirement = "Create a function that finds the longest palindromic substring in a given string"
result = code_generation_workflow(requirement)

=== PATTERN 6: Code Generation and Refinement ===

=== Pseudocode ===
1. Define a function called findLongestPalindromicSubstring that takes a string as input.

2. Initialize a variable called longestPalindrome to an empty string.

3. Create a nested loop to iterate through all possible substrings in the input string:
   - Outer loop: for i from 0 to the length of the input string
   - Inner loop: for j from i+1 to the length of the input string

4. Inside the nested loop, extract the current substring using string slicing:
   - substring = input_string[i:j+1]

5. Check if the current substring is a palindrome:
   - Create a helper function called isPalindrome that takes a string as input and returns a boolean value indicating whether it is a palindrome or not.
   - If isPalindrome(substring) returns true and the length of the current substring is greater than the length of the longestPalindrome, update longestPalindrome to be the current substring.

6. After the nested loop completes,

## Pattern 7: Multimodal and Multi-step Reasoning

This pattern demonstrates processing information from multiple modalities (image + text + structured data).

**Use Case**: Invoice processing, document understanding, image-text analysis

**Note**: This example uses GPT-4 Vision capabilities. Make sure you have access to vision-enabled models.

In [16]:
# Pattern 7: Multimodal and Multi-step Reasoning
# 3-step chain: extract image text -> link labels -> interpret with table

import base64
from pathlib import Path

# For vision capabilities, use GPT-4 Vision
vision_llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Load invoice image
image_path = Path("../assets/invoice_sample.png")

# Step 1: Extract text from image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def extract_text_from_image(image_path):
    base64_image = encode_image(image_path)
    
    prompt = ChatPromptTemplate.from_messages([
        ("user", [
            {"type": "text", "text": "Extract all text from this invoice image. List each piece of information clearly."},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
        ])
    ])
    
    chain = prompt | vision_llm | parser
    return chain.invoke({})

# Step 2: Link extracted text with labels
prompt_link_labels = ChatPromptTemplate.from_template(
    """From the extracted invoice text, identify and label these key fields:

Extracted text:
{extracted_text}

Create a labeled structure:
- Invoice Number: 
- Date:
- Customer:
- Items:
- Total Amount:
"""
)

# Step 3: Interpret using business rules table
prompt_interpret = ChatPromptTemplate.from_template(
    """Using this labeled invoice data and business rules, determine the required action:

Labeled Data:
{labeled_data}

Business Rules Table:
| Total Amount | Customer Type | Action |
|--------------|---------------|--------|
| < $1000      | Any           | Auto-approve |
| $1000-$5000  | Existing      | Manager review |
| $1000-$5000  | New           | Director review |
| > $5000      | Any           | Director approval required |

Determine:
1. Which rule applies
2. Required action
3. Any special notes or flags
"""
)

# Build multimodal workflow
link_chain = prompt_link_labels | llm | parser  
interpret_chain = prompt_interpret | llm | parser

def multimodal_workflow(image_path):
    # Step 1: Extract text from image
    print("\n=== Step 1: Extracting text from invoice image ===")
    extracted_text = extract_text_from_image(image_path)
    print(extracted_text)
    
    # Step 2: Link with labels
    print("\n=== Step 2: Linking extracted text with labels ===")
    labeled_data = link_chain.invoke({"extracted_text": extracted_text})
    print(labeled_data)
    
    # Step 3: Interpret with business rules
    print("\n=== Step 3: Interpreting with business rules ===")
    final_decision = interpret_chain.invoke({"labeled_data": labeled_data})
    print(final_decision)
    
    return final_decision

print("=== PATTERN 7: Multimodal and Multi-step Reasoning ===")
if image_path.exists():
    result = multimodal_workflow(image_path)
else:
    print(f"Image not found at {image_path}. Please ensure the invoice_sample.png exists in the assets folder.")

=== PATTERN 7: Multimodal and Multi-step Reasoning ===

=== Step 1: Extracting text from invoice image ===
**Invoice Information:**

- **Title:** ACME Corp Invoice

- **Company Name:** ACME Corp
- **Address:** 456 Business Avenue, Springfield, IL 62701
- **Contact Email:** info@gmail.com

- **Invoice Number:** #12345
- **Date:** 2024-01-15

**Bill To:**

- **Name:** John Smith
- **Address:** 123 Main Street, New York, NY 10001

**Items:**

1. **Item:** Laptop Computer
   - **Quantity:** 2
   - **Unit Price:** $1,200
   - **Total:** $2,400

2. **Item:** Wireless Mouse
   - **Quantity:** 5
   - **Unit Price:** $25
   - **Total:** $125

**Summary:**

- **Subtotal:** $2,525
- **Tax (8%):** $202
- **Total Amount Due:** $2,727

**Footer:**

- **Message:** Thank you for your business! Payment due within 30 days.

=== Step 2: Linking extracted text with labels ===
- Invoice Number: #12345
- Date: 2024-01-15
- Customer:
  - Name: John Smith
  - Address: 123 Main Street, New York, NY 10001
- Ite

## Conclusion

This notebook demonstrated 7 comprehensive prompt chaining patterns:

1. **Information Processing Workflows** - Multi-step document analysis pipeline
2. **Complex Query Answering** - Breaking down and synthesizing research questions  
3. **Data Extraction and Transformation** - Iterative extraction with validation
4. **Content Generation Workflows** - Progressive content creation
5. **Conversational Agents with State** - Context-aware multi-turn dialogues
6. **Code Generation and Refinement** - Iterative code development
7. **Multimodal and Multi-step Reasoning** - Processing images, text, and structured data

Each pattern demonstrates how prompt chaining enables complex AI workflows by breaking tasks into manageable, sequential steps.